In [3]:
%pip install -q jsonstat.py


error: externally-managed-environment

× This environment is externally managed
╰─> To install Python packages system-wide, try 'pacman -S
    python-xyz', where xyz is the package you are trying to
    install.
    
    If you wish to install a non-Arch-packaged Python package,
    create a virtual environment using 'python -m venv path/to/venv'.
    Then use path/to/venv/bin/python and path/to/venv/bin/pip.
    
    If you wish to install a non-Arch packaged Python application,
    it may be easiest to use 'pipx install xyz', which will manage a
    virtual environment for you. Make sure you have python-pipx
    installed via pacman.

note: If you believe this is a mistake, please contact your Python installation or OS distribution provider. You can override this, at the risk of breaking your Python installation or OS, by passing --break-system-packages.
hint: See PEP 668 for the detailed specification.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import requests
import json

# Paesi di interesse
paesi = ['IT', 'FR', 'DE', 'ES', 'EL', 'PT', 'NL', 'PL', 'CZ']

# Variabili macroeconomiche di interesse (codici Eurostat - da verificare!)
variabili = {
    'occupazione': 'EMP',  # Esempio: potrebbe essere necessario specificare ulteriormente (es. tasso di occupazione)
    'retribuzioni': 'EARN', # Esempio: potrebbe essere necessario specificare ulteriormente (es. retribuzioni medie lorde)
    'tasso_disoccupazione': 'UNE' # Esempio: tasso di disoccupazione
}

# Nuova base URL per la SDMX 2.1 REST API di Eurostat
base_url = "https://ec.europa.eu/eurostat/api/dissemination/sdmx/2.1/data/"

# Funzione per scaricare i dati da Eurostat
def scarica_dati_eurostat(dataset_code, params=None):
    url = f"{base_url}{dataset_code}/DSD?"  # Richiediamo prima la struttura del dataset (DSD)
    if params:
        url += "&".join([f"{key}={value}" for key, value in params.items()]) + "&format=JSON"
    else:
        url += "format=JSON"
    try:
        response = requests.get(url)
        response.raise_for_status()
        dsd_data = response.json()

        # Costruisci l'URL per i dati effettivi
        data_url = f"{base_url}{dataset_code}/?"
        if params:
            data_url += "&".join([f"{key}={value}" for key, value in params.items()]) + "&format=JSON"
        else:
            data_url += "format=JSON"

        data_response = requests.get(data_url)
        data_response.raise_for_status()
        data = data_response.json()
        return data, dsd_data # Restituiamo sia i dati che la struttura
    except requests.exceptions.RequestException as e:
        print(f"Errore nella richiesta a Eurostat: {e}")
        return None, None

# Funzione per processare i dati JSON e creare un DataFrame
def processa_dati_eurostat(data, dsd_data, variabile):
    if not data or 'structure' not in data or 'dataSets' not in data or not dsd_data or 'structure' not in dsd_data:
        return None

    # Estrai le informazioni sulla struttura (dimensioni, codici, ecc.) dal DSD
    struttura = dsd_data['structure']
    dimensioni = {dim['id']: {item['id']: item['name'] for item in dim['values']} for dim in struttura['dimensions']['allDimensions']}

    # Estrai i dati osservazionali
    dati_osservazionali = data['dataSets'][0]['observations']

    records = []
    for key, values in dati_osservazionali.items():
        record = {}
        key_parts = key.split('.')
        dim_index = 0
        for dim_id, dim_values in dimensioni.items():
            if dim_index < len(key_parts):
                record[dim_id] = dim_values.get(key_parts[dim_index], key_parts[dim_index]) # Gestisci chiavi non trovate
                dim_index += 1
        record['valore'] = values[0]
        records.append(record)

    df = pd.DataFrame(records)
    df['variabile'] = variabile
    return df

# Esempio di come potresti scaricare i dati per il tasso di disoccupazione
dataset_tasso_disoccupazione = "LFS" # Questo è un codice di esempio, dovrai trovare quello corretto
parametri_disoccupazione = {
    'geo': ','.join(paesi),
    'freq': 'M', # Frequenza mensile (potresti voler 'A' per annuale)
    'sinceTime': '1950' # Potrebbe non essere supportato da tutti i dataset
}
dati_disoccupazione_json, dsd_disoccupazione_json = scarica_dati_eurostat(dataset_tasso_disoccupazione, parametri_disoccupazione)
df_disoccupazione = processa_dati_eurostat(dati_disoccupazione_json, dsd_disoccupazione_json, 'tasso_disoccupazione')

# Fai lo stesso per le altre variabili (occupazione e retribuzioni),
# identificando i dataset e i parametri appropriati.

# Infine, potresti voler unire i DataFrame risultanti in un unico DataFrame più grande.
if df_disoccupazione is not None:
    print("DataFrame del tasso di disoccupazione (anteprima):")
    print(df_disoccupazione.head())
else:
    print("Impossibile scaricare o processare i dati sul tasso di disoccupazione.")

Errore nella richiesta a Eurostat: 404 Client Error: Not Found for url: https://ec.europa.eu/eurostat/api/dissemination/sdmx/2.1/data/LFS/DSD?geo=IT,FR,DE,ES,EL,PT,NL,PL,CZ&freq=M&sinceTime=1950&format=JSON
Impossibile scaricare o processare i dati sul tasso di disoccupazione.
